<a href="https://colab.research.google.com/github/proygit/DingNet/blob/master/DMT_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

# Change the working directory to the desired folder
os.chdir('/content/drive/MyDrive/dmt-data')

# List the contents of the current working directory
contents = os.listdir()

# Print the contents
print(contents)
# Read in the Excel file
data_df = pd.read_excel('ODI-2023.xlsx')

# Create a copy of the DataFrame
df_cp = data_df.copy()

# Get the shape of the DataFrame
print(df_cp.shape)
df_cp.columns
df_cp.describe()


['ODI-2023.xlsx', 'DMT.csv']
(305, 17)


<ipython-input-20-086b583ed9a4>:25: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_cp.describe()


,Tijdstempel,What programme are you in?,Have you taken a course on machine learning?,Have you taken a course on information retrieval?,Have you taken a course on statistics?,Have you taken a course on databases?,What is your gender?,I have used ChatGPT to help me with some of my study assignments,When is your birthday (date)?,How many students do you estimate there are in the room?,Did you stand up to come to your previous answer ?,What is your stress level (0-100)?,How many hours per week do you do sports (in whole hours)?,Give a random number,Time you went to bed Yesterday,What makes a good day for you (1)?,What makes a good day for you (2)?
count,305,305,305,305.0,305,305,305,305,305,305.0,305,304.0,305.0,305.0,305,305,304
unique,302,129,3,3.0,3,3,6,3,292,107.0,3,76.0,36.0,137.0,125,209,246
top,2023-04-04 12:11:08.480000,AI,yes,0.0,mu,ja,male,yes,14-05-1999,400.0,no,50.0,4.0,7.0,23:30,Sun,Good food
freq,2,43,221,189.0,259,168,154,192,3,39.0,276,21.0,37.0,24.0,19,15,10
first,2023-04-04 12:08:33.342000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2023-04-04 12:28:05.558000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_cp.to_csv('DMT.csv',index =False, sep=',', header=True)
# Print the first few rows of the DataFrame
df_csv = pd.read_csv('DMT.csv')
df_csv_ver1 =df_csv[[#'Tijdstempel', 
        #'What programme are you in?',
       #'Have you taken a course on machine learning?',
       #'Have you taken a course on information retrieval?',
       #'Have you taken a course on statistics?',
       #'Have you taken a course on databases?', 
        'What is your gender?',
       'I have used ChatGPT to help me with some of my study assignments ',
      # 'When is your birthday (date)?',
       #'How many students do you estimate there are in the room?',
       #'Did you stand up to come to your previous answer    ?',
       #'What is your stress level (0-100)?',
       #'How many hours per week do you do sports (in whole hours)? ',
       #'Give a random number', 
        #'Time you went to bed Yesterday',
       #'What makes a good day for you (1)?',
       #'What makes a good day for you (2)?'
       ]].iloc[:, :17]
df_csv_ver1


,What is your gender?,I have used ChatGPT to help me with some of my study assignments
0,gender fluid,yes
1,male,not willing to say
2,female,no
3,male,yes
4,male,yes
...,...,...
300,male,no
301,female,no
302,gender fluid,no
303,male,not willing to say


In [ ]:
# Rename the column since its too long 
df_csv_ver1 = df_csv_ver1.rename(columns={'I have used ChatGPT to help me with some of my study assignments ': 'Used ChatGPT?'})
df_csv_ver1 = df_csv_ver1.rename(columns={'What is your gender?': 'Gender'})
#check values of Gender if there are Male or Female or anything else
df_csv_ver1['Gender'].value_counts()
# if the values are not male or female convert them to NaN
mask = ~ df_csv_ver1['Gender'].isin(['male', 'female'])
df_csv_ver1.loc[mask, 'Gender'] = np.nan
# drop the NaN row values 
df_csv_ver1.dropna(inplace=True)

df_csv_ver1

#male =1, female =0 
#not willing to say = 1, no = 0,yes = 2

,Gender,Used ChatGPT?
1,male,not willing to say
2,female,no
3,male,yes
4,male,yes
5,female,yes
...,...,...
299,male,not willing to say
300,male,no
301,female,no
303,male,not willing to say


In [ ]:

#Convert categorial to numeric values
le = LabelEncoder()
df_csv_ver1['Gender'] = le.fit_transform(df_csv_ver1['Gender'])
df_csv_ver1['Used ChatGPT?'] = le.fit_transform(df_csv_ver1['Used ChatGPT?'])

In [ ]:
df_csv_ver1


,Gender,Used ChatGPT?
1,1,1
2,0,0
3,1,2
4,1,2
5,0,2
...,...,...
299,1,1
300,1,0
301,0,0
303,1,1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# standadization is not needed since the data is clean and their is no noise.
#Split the data set in train and test
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_csv_ver1[['Gender']], df_csv_ver1['Used ChatGPT?'], test_size=0.2, random_state=42)

# use logistic regression 
# Train a Logistic Regression model on the training set
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

# Evaluate the performance of the model on the test set
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6206896551724138


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_csv_ver1[['Gender']], df_csv_ver1['Used ChatGPT?'], test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a KNN model on the training set
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, y_train)

# Evaluate the performance of the model on the test set
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Accuracy: 0.6206896551724138


# Update the data set to include more features and then apply the model again

In [ ]:
df_cp.to_csv('DMT.csv',index =False, sep=',', header=True)
# Print the first few rows of the DataFrame
df_csv_ver2 = pd.read_csv('DMT.csv')
df_csv_ver2 =df_csv[[#'Tijdstempel', 
        'What programme are you in?',
       'Have you taken a course on machine learning?',
       'Have you taken a course on information retrieval?',
       'Have you taken a course on statistics?',
       'Have you taken a course on databases?', 
        'What is your gender?',
       'I have used ChatGPT to help me with some of my study assignments ',
      # 'When is your birthday (date)?',
       #'How many students do you estimate there are in the room?',
       #'Did you stand up to come to your previous answer    ?',
       #'What is your stress level (0-100)?',
       #'How many hours per week do you do sports (in whole hours)? ',
       #'Give a random number', 
        #'Time you went to bed Yesterday',
       #'What makes a good day for you (1)?',
       #'What makes a good day for you (2)?'
       ]].iloc[:, :17]
df_csv_ver2

,What programme are you in?,Have you taken a course on machine learning?,Have you taken a course on information retrieval?,Have you taken a course on statistics?,Have you taken a course on databases?,What is your gender?,I have used ChatGPT to help me with some of my study assignments
0,Masters - AI,yes,0,mu,nee,gender fluid,yes
1,Artificial Intelligence,no,0,sigma,nee,male,not willing to say
2,AI,no,0,mu,nee,female,no
3,Data mining,yes,1,mu,ja,male,yes
4,Quantitative Risk Management,no,0,mu,nee,male,yes
...,...,...,...,...,...,...,...
300,Exchange,no,0,unknown,ja,male,no
301,Bioinformatics,yes,unknown,mu,nee,female,no
302,AI,yes,0,mu,ja,gender fluid,no
303,AI,yes,0,mu,nee,male,not willing to say


#rename the columns and clean the data

In [ ]:
#Remane the columns to a smaller name
df_csv_ver2 = df_csv_ver2.rename(columns={'What programme are you in?': 'Study Track','Have you taken a course on machine learning?':'taken ML before?'})
df_csv_ver2 = df_csv_ver2.rename(columns={'Have you taken a course on information retrieval?': 'taken IR before?','Have you taken a course on statistics?':'taken Statistics before?','Have you taken a course on databases?':'taken DB before?'})
df_csv_ver2= df_csv_ver2.rename(columns={'I have used ChatGPT to help me with some of my study assignments ': 'Used ChatGPT?'})
df_csv_ver2 = df_csv_ver2.rename(columns={'What is your gender?': 'Gender'})

df_csv_ver2


,Study Track,taken ML before?,taken IR before?,taken Statistics before?,taken DB before?,Gender,Used ChatGPT?
0,Masters - AI,yes,0,mu,nee,gender fluid,yes
1,Artificial Intelligence,no,0,sigma,nee,male,not willing to say
2,AI,no,0,mu,nee,female,no
3,Data mining,yes,1,mu,ja,male,yes
4,Quantitative Risk Management,no,0,mu,nee,male,yes
...,...,...,...,...,...,...,...
300,Exchange,no,0,unknown,ja,male,no
301,Bioinformatics,yes,unknown,mu,nee,female,no
302,AI,yes,0,mu,ja,gender fluid,no
303,AI,yes,0,mu,nee,male,not willing to say


In [ ]:
#check values of Gender if there are Male or Female or anything else
# if the values are not male or female convert them to NaN
# Define the valid study tracks
study_tracks = ['Business Analytics','Bioinformatics','Bioinformatics & Systems Biology','Econometrics','Artificial Intelligence', 'AI', 'Masters - AI','M.S. Artificial Intelligence','Computational Science','BA','Econometrics and Data Science','CS']

# Convert all study tracks in the list to lowercase
study_tracks_lower = [s.lower() for s in study_tracks]

# Iterate over each row in the dataframe
for index, row in df_csv_ver2.iterrows():
    # Convert the study track to lowercase
    study_track_lower = str(row['Study Track']).lower()
    
    # If the study track is not in the list, mark it as NaN
    if study_track_lower not in study_tracks_lower:
        df_csv_ver2.loc[index, 'Study Track'] = np.nan

df_csv_ver2
#check how many we lose by having Nan
df_csv_ver2['Study Track'].isna().value_counts()
#after cleaningf study track we have 136 data of students in different tracks.
#So drop all other row in the csv for other tracks which are NaN
df_csv_ver2.dropna(inplace=True)
df_csv_ver2['Study Track'].isna().value_counts()
df_csv_ver2
# Replace "unknown" with 0 in 'taken IR before?' column
df_csv_ver2['taken IR before?'] = df_csv_ver2['taken IR before?'].replace('unknown', 0)
# Replace "unknown" with "sigma" in 'taken Statistics before?' column
df_csv_ver2['taken Statistics before?'] = df_csv_ver2['taken Statistics before?'].replace('unknown', 'sigma')
#Apply LabelEncoding values 1=yes,mu and ja AND 0 = no,sigma,nee
le = LabelEncoder()
# Encode the 'yes' and 'no' values in the 'taken IR before?' column as binary 1 and 0, respectively
df_csv_ver2['taken ML before?'] = le.fit_transform(df_csv_ver2['taken ML before?'])
df_csv_ver2['taken Statistics before?'] = le.fit_transform(df_csv_ver2['taken Statistics before?'])
df_csv_ver2['taken DB before?'] = le.fit_transform(df_csv_ver2['taken DB before?'])
#check after cleaning
df_csv_ver2
#clean for Gender and UsedGPT

mask = ~ df_csv_ver2['Gender'].isin(['male', 'female'])
df_csv_ver2.loc[mask, 'Gender'] = np.nan
# drop the NaN row values 
df_csv_ver2.dropna(inplace=True)
df_csv_ver2['Gender'] = le.fit_transform(df_csv_ver2['Gender'])
df_csv_ver2['Used ChatGPT?'] = le.fit_transform(df_csv_ver2['Used ChatGPT?'])
df_csv_ver2

,Study Track,taken ML before?,taken IR before?,taken Statistics before?,taken DB before?,Gender,Used ChatGPT?
1,Artificial Intelligence,0,0,1,1,1,1
2,AI,0,0,0,1,0,0
5,Business Analytics,1,0,0,0,0,2
6,M.S. Artificial Intelligence,1,0,0,0,1,2
7,Artificial Intelligence,1,1,0,0,1,2
...,...,...,...,...,...,...,...
294,Artificial Intelligence,0,0,0,0,1,2
296,Bioinformatics & Systems Biology,1,0,0,1,0,0
298,Bioinformatics,1,0,0,1,1,2
301,Bioinformatics,1,0,0,1,0,0


In [ ]:
df_csv_ver2.describe
df_csv_ver3 = df_csv_ver2.drop("Study Track", axis=1).copy()
df_csv_ver3.shape


(139, 6)

In [ ]:
# Select features and target variable
X = df_csv_ver3[['taken ML before?', 'taken IR before?', 'taken Statistics before?', 'taken DB before?','Gender']]
y = df_csv_ver3['Used ChatGPT?']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Predictions on testing set
y_pred = knn.predict(X_test)

# Evaluation
accuracy = knn.score(X_test, y_test)
print('Accuracy:', accuracy)

Accuracy: 0.5476190476190477


In [ ]:
# Import libraries
from sklearn.linear_model import LogisticRegression

# Select features and target variable
X = df_csv_ver3[['taken ML before?', 'taken IR before?', 'taken Statistics before?', 'taken DB before?','Gender']]
y = df_csv_ver3['Used ChatGPT?']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Predictions on testing set
y_pred = log_reg.predict(X_test)

# Evaluation
accuracy = log_reg.score(X_test, y_test)
print('Accuracy:', accuracy)


Accuracy: 0.6904761904761905


In [ ]:
from sklearn.svm import SVC

# Select features and target variable
X = df_csv_ver3[['taken ML before?', 'taken IR before?', 'taken Statistics before?', 'taken DB before?','Gender']]
y = df_csv_ver3['Used ChatGPT?']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# SVM classifier
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

# Predictions on testing set
y_pred = svm.predict(X_test)

# Evaluation
accuracy = svm.score(X_test, y_test)
print('Accuracy:', accuracy)


Accuracy: 0.7380952380952381


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Select features and target variable
X = df_csv_ver3[['taken ML before?', 'taken IR before?', 'taken Statistics before?', 'taken DB before?']]
y = df_csv_ver3['Used ChatGPT?']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Decision Tree classifier
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

# Predictions on testing set
y_pred = dt.predict(X_test)

# Evaluation
accuracy = dt.score(X_test, y_test)
print('Accuracy:', accuracy)


Accuracy: 0.6904761904761905


In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Instantiate and fit KNN model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Make predictions on test set
y_pred = knn.predict(X_test)

# Calculate accuracy score
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.6904761904761905
